In [1]:
import scrapy
from scrapy.crawler import CrawlerRunner
from crochet import setup, wait_for
import json
import logging
import re
import seaborn as sns
import unihandecode
import pipe
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
from pipe import select

In [2]:
# inicializar la librería crochet
setup()

In [3]:
from itemadapter import ItemAdapter

class JsonWriterPipeline:

    def open_spider(self, spider):
        self.file = open('tokens.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(ItemAdapter(item).asdict()) + "\n"
        self.file.write(line)
        return item

In [4]:
class WikiSpider(scrapy.Spider):
    name = 'wiki'
    allowed_domains = ['es.wikipedia.org']
    start_urls = ['https://en.wikipedia.org/wiki/Wikipedia:Featured_articles']

    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1},
        'FEED_FORMAT': 'json',
        'FEED_URI': 'token_list.json'
    }

    def parse(self, response):
        i = 0
        for link in response.css('.featured_article_metadata > a'):
            yield response.follow(link.attrib.get('href'), callback=self.parse_article_data)
            i += 1
            if i == 1: # Solo leer el primer link
                break

    def parse_article_data(self, response):

        def conv(org_list, seperator=' '):
            return seperator.join(org_list).lower()

        contenido = response.xpath('/html/body')
        for parrafo in contenido:
            lista_parrafo_grande = parrafo.xpath('//p//text()').getall()
            sin_unicode = list(lista_parrafo_grande | select(lambda x: unihandecode.unidecode(x)))
            sin_especial = [re.sub('[^a-zA-Z0-9]+|[\]\[\b\d+\b]', ' ', _) for _ in sin_unicode]
            sin_espacios = list(sin_especial | select(lambda x: x.strip()))
            sin_blancos = [x for x in sin_espacios if x]
            sin_dobles = [re.sub(r' +', ' ', string) for string in sin_blancos]
            parrafo_grande = conv(sin_dobles)

        yield {
                'texto': parrafo_grande
            }

In [5]:
def run_spider():
    crawler = CrawlerRunner()
    d = crawler.crawl(WikiSpider)
    return d

In [6]:
run_spider()

In [ ]:
# !cat tokens.jl
# https://bit.ly/3NOEsmL

In [ ]:
# !tail -n 2 token_list.json

In [7]:
f = open('token_list.json')
data = json.load(f)
f.close()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
len(data)

In [ ]:
type(data)

In [ ]:
data

In [ ]:
lista = ' '.join(str(e) for e in data)

In [ ]:
type(lista)

In [ ]:
len(lista)

In [ ]:
lista[0:50]

In [ ]:
cadena = lista.split(' ', 1)[1]

In [ ]:
type(cadena)

In [ ]:
tokens = re.split('\W+', cadena)

In [ ]:
len(tokens)

In [ ]:
print(tokens[0:10])

In [ ]:
tokens[0]

In [ ]:
del tokens[0]

In [ ]:
len(tokens)

In [ ]:
print(tokens[:100])

In [ ]:
len(tokens)

In [ ]:
gen = (i*10**exp for exp in range(1, 6) for i in range(1, 11))
rangos = list(dict.fromkeys(list(gen)))

In [ ]:
print(rangos)

In [ ]:
listm = []
for i in rangos:
    listm.append(tokens[0:i])

In [ ]:
print(listm[0:2])

In [ ]:
eje = []
for i in listm:
    eje.append(len(set(i)))

In [ ]:
print(eje[0:5])

In [ ]:
len(eje)

In [ ]:
d = {'tokens': rangos, 'vocabulario': eje}

In [ ]:
df = pd.DataFrame(d)

In [ ]:
df.tail()

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
sns.scatterplot(data=df, x="tokens", y="vocabulario", ax = ax, s = 150)